# 🧠 NOTEBOOK 5 : DEEP LEARNING POUR LA CLASSIFICATION D'IMAGES

## Objectifs
1. **Classifier les produits** par catégorie à partir de leurs images via Deep Learning
2. **Entraîner plusieurs architectures CNN** :
   - CNN custom from scratch
   - **Transfer Learning** avec fine-tuning (ResNet50, EfficientNetB0)
3. **Data augmentation** pour améliorer la généralisation
4. **Évaluer les performances** avec métriques complètes
5. **Récapitulatif final** : Résultats modèle texte vs modèle images

**7 catégories** : Baby Care, Beauty and Personal Care, Computers, Home Decor & Festive Needs, Home Furnishing, Kitchen & Dining, Watches

**Approches** :
- CNN custom (architecture légère)
- ResNet50 (fine-tuning)
- EfficientNetB0 (fine-tuning)

## 1. IMPORTS ET CONFIGURATION

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import os
warnings.filterwarnings('ignore')

# Images
from PIL import Image
import cv2

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Model
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.layers import (
    Dense, Dropout, GlobalAveragePooling2D, BatchNormalization,
    Conv2D, MaxPooling2D, Flatten, Activation
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Sklearn pour métriques
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)

# Config matplotlib
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
%matplotlib inline

print(f"✅ TensorFlow version : {tf.__version__}")
print(f"✅ Keras version : {keras.__version__}")
print(f"✅ GPU disponible : {len(tf.config.list_physical_devices('GPU')) > 0}")

# Config GPU (si disponible)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU configuré : memory growth enabled")
    except RuntimeError as e:
        print(f"⚠️ Erreur config GPU : {e}")

In [ ]:
# Configuration des chemins
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "Data"
IMAGES_DIR = DATA_DIR / "Images"
RESULTS_DIR = BASE_DIR / "results"
MODELS_DIR = RESULTS_DIR / "models"
MODELS_DIR.mkdir(exist_ok=True, parents=True)

# Fichiers
METADATA_FILE = DATA_DIR / "data_images_corrected.csv"

print(f"📂 Images : {IMAGES_DIR}")
print(f"📂 Métadonnées : {METADATA_FILE}")
print(f"📂 Modèles : {MODELS_DIR}")

# Hyperparamètres globaux
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50
RANDOM_STATE = 42

print(f"\n⚙️ Hyperparamètres :")
print(f"   Image size : {IMG_SIZE}")
print(f"   Batch size : {BATCH_SIZE}")
print(f"   Max epochs : {EPOCHS}")

## 2. CHARGEMENT ET PRÉPARATION DES DONNÉES

In [ ]:
# Charger les métadonnées
df = pd.read_csv(METADATA_FILE)

print(f"✅ Dataset chargé : {df.shape[0]} images")
print(f"\n📊 Distribution des catégories :")
print(df['main_category'].value_counts())

# Encoder les labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['main_category'])
n_classes = len(le.classes_)

print(f"\n✅ {n_classes} classes encodées :")
for idx, cat in enumerate(le.classes_):
    print(f"   {idx} → {cat}")

In [ ]:
# Vérifier la présence des images
print("🔍 Vérification de la présence des images...\n")

missing_count = 0
valid_indices = []

for idx, row in df.iterrows():
    img_path = IMAGES_DIR / row['image']
    if img_path.exists():
        valid_indices.append(idx)
    else:
        missing_count += 1

print(f"✅ Images trouvées : {len(valid_indices)}/{len(df)}")
print(f"⚠️ Images manquantes : {missing_count}")

# Garder uniquement les images présentes
if missing_count > 0:
    df = df.loc[valid_indices].reset_index(drop=True)
    print(f"\n✅ Dataset filtré : {len(df)} images")

In [ ]:
# Split train/validation/test : 70/15/15
train_df, temp_df = train_test_split(
    df, test_size=0.3, random_state=RANDOM_STATE, stratify=df['label_encoded']
)

val_df, test_df = train_test_split(
    temp_df, test_size=0.5, random_state=RANDOM_STATE, stratify=temp_df['label_encoded']
)

print("✅ Split train/val/test (70/15/15) :")
print(f"   Train : {len(train_df)} images ({len(train_df)/len(df)*100:.1f}%)")
print(f"   Val   : {len(val_df)} images ({len(val_df)/len(df)*100:.1f}%)")
print(f"   Test  : {len(test_df)} images ({len(test_df)/len(df)*100:.1f}%)")

# Vérifier la distribution
print(f"\n📊 Distribution dans le test set :")
for cat in le.classes_:
    count = (test_df['main_category'] == cat).sum()
    print(f"   {cat:<35} : {count:3d} images")

## 3. DATA GENERATORS AVEC AUGMENTATION

In [ ]:
# Data augmentation pour le training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.15,
    fill_mode='nearest'
)

# Validation et test : seulement rescaling
val_test_datagen = ImageDataGenerator(rescale=1./255)

print("✅ ImageDataGenerators créés")
print("\n🔄 Augmentation (train) :")
print("   - Rotation : ±20°")
print("   - Translation : ±20%")
print("   - Flip horizontal")
print("   - Zoom : ±20%")
print("   - Shear : ±15%")
print("\n📏 Val/Test : rescaling uniquement")

In [ ]:
# Fonction pour créer un générateur depuis un DataFrame
def create_generator(datagen, dataframe, batch_size=BATCH_SIZE, shuffle=True):
    """
    Crée un générateur custom qui charge les images depuis les chemins du DataFrame
    """
    def generator():
        indices = np.arange(len(dataframe))
        if shuffle:
            np.random.shuffle(indices)
        
        for start_idx in range(0, len(dataframe), batch_size):
            batch_indices = indices[start_idx:start_idx + batch_size]
            batch_df = dataframe.iloc[batch_indices]
            
            batch_images = []
            batch_labels = []
            
            for _, row in batch_df.iterrows():
                img_path = IMAGES_DIR / row['image']
                try:
                    img = keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
                    img_array = keras.preprocessing.image.img_to_array(img)
                    batch_images.append(img_array)
                    batch_labels.append(row['label_encoded'])
                except Exception as e:
                    continue
            
            if batch_images:
                X = np.array(batch_images)
                y = to_categorical(batch_labels, num_classes=n_classes)
                
                # Appliquer l'augmentation
                for i in range(len(X)):
                    X[i] = datagen.random_transform(X[i])
                    X[i] = datagen.standardize(X[i])
                
                yield X, y
    
    return generator

# Créer les générateurs
train_gen = create_generator(train_datagen, train_df, shuffle=True)
val_gen = create_generator(val_test_datagen, val_df, shuffle=False)
test_gen = create_generator(val_test_datagen, test_df, shuffle=False)

# Calculer steps
train_steps = len(train_df) // BATCH_SIZE
val_steps = len(val_df) // BATCH_SIZE
test_steps = len(test_df) // BATCH_SIZE

print(f"\n✅ Générateurs créés")
print(f"   Train steps per epoch : {train_steps}")
print(f"   Val steps per epoch   : {val_steps}")
print(f"   Test steps            : {test_steps}")

## 4. MODÈLE 1 : CNN CUSTOM (FROM SCRATCH)

### 4.1 Architecture

In [ ]:
def build_custom_cnn(input_shape=(224, 224, 3), num_classes=7):
    """
    CNN custom simple mais efficace
    """
    model = models.Sequential([
        # Block 1
        Conv2D(32, (3, 3), padding='same', input_shape=input_shape),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(32, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Block 2
        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Block 3
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.3),
        
        # Dense layers
        Flatten(),
        Dense(256),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),
        Dense(128),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.4),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Créer le modèle
cnn_model = build_custom_cnn(input_shape=(*IMG_SIZE, 3), num_classes=n_classes)

print("✅ CNN custom créé")
print(f"\n📊 Résumé de l'architecture :")
cnn_model.summary()

### 4.2 Compilation et entraînement

In [ ]:
# Compiler le modèle
cnn_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Modèle compilé")
print("   Optimizer : Adam (lr=0.001)")
print("   Loss : categorical_crossentropy")
print("   Metrics : accuracy")

In [ ]:
# Callbacks
cnn_callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        str(MODELS_DIR / 'cnn_custom_best.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("✅ Callbacks configurés :")
print("   - EarlyStopping (patience=10)")
print("   - ReduceLROnPlateau (factor=0.5, patience=5)")
print("   - ModelCheckpoint (best val_accuracy)")

In [ ]:
# Entraînement
print("\n🔄 Entraînement CNN custom...")
print(f"⏳ {EPOCHS} epochs max (early stopping activé)\n")

history_cnn = cnn_model.fit(
    tf.data.Dataset.from_generator(
        train_gen,
        output_signature=(
            tf.TensorSpec(shape=(None, *IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, n_classes), dtype=tf.float32)
        )
    ).repeat(),
    steps_per_epoch=train_steps,
    epochs=EPOCHS,
    validation_data=tf.data.Dataset.from_generator(
        val_gen,
        output_signature=(
            tf.TensorSpec(shape=(None, *IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, n_classes), dtype=tf.float32)
        )
    ).repeat(),
    validation_steps=val_steps,
    callbacks=cnn_callbacks,
    verbose=1
)

print("\n✅ Entraînement CNN terminé")

### 4.3 Courbes d'apprentissage

In [ ]:
# Visualisation des courbes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss
ax1.plot(history_cnn.history['loss'], label='Train Loss', linewidth=2)
ax1.plot(history_cnn.history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=11, fontweight='bold')
ax1.set_ylabel('Loss', fontsize=11, fontweight='bold')
ax1.set_title('CNN Custom — Loss', fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# Accuracy
ax2.plot(history_cnn.history['accuracy'], label='Train Accuracy', linewidth=2)
ax2.plot(history_cnn.history['val_accuracy'], label='Val Accuracy', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=11, fontweight='bold')
ax2.set_ylabel('Accuracy', fontsize=11, fontweight='bold')
ax2.set_title('CNN Custom — Accuracy', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Meilleures métriques
best_epoch = np.argmax(history_cnn.history['val_accuracy'])
best_val_acc = history_cnn.history['val_accuracy'][best_epoch]
best_val_loss = history_cnn.history['val_loss'][best_epoch]

print(f"\n🏆 Meilleures performances (epoch {best_epoch + 1}) :")
print(f"   Val Accuracy : {best_val_acc:.4f}")
print(f"   Val Loss     : {best_val_loss:.4f}")

### 4.4 Évaluation sur le test set

In [ ]:
# Évaluation
print("🔄 Évaluation CNN custom sur le test set...\n")

test_loss_cnn, test_acc_cnn = cnn_model.evaluate(
    tf.data.Dataset.from_generator(
        test_gen,
        output_signature=(
            tf.TensorSpec(shape=(None, *IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, n_classes), dtype=tf.float32)
        )
    ),
    steps=test_steps
)

print(f"\n✅ CNN Custom — Test Set :")
print(f"   Loss     : {test_loss_cnn:.4f}")
print(f"   Accuracy : {test_acc_cnn:.4f} ({test_acc_cnn*100:.2f}%)")

## 5. MODÈLE 2 : TRANSFER LEARNING — RESNET50

### 5.1 Architecture avec fine-tuning

In [ ]:
def build_resnet50_model(input_shape=(224, 224, 3), num_classes=7, trainable_layers=30):
    """
    ResNet50 pré-entraîné avec fine-tuning
    """
    # Charger ResNet50 pré-entraîné (ImageNet)
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Geler toutes les couches sauf les N dernières
    for layer in base_model.layers[:-trainable_layers]:
        layer.trainable = False
    
    # Ajouter les couches de classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=output)
    
    return model, base_model

# Créer le modèle
resnet_model, resnet_base = build_resnet50_model(
    input_shape=(*IMG_SIZE, 3),
    num_classes=n_classes,
    trainable_layers=30
)

# Compter les couches gelées/entraînables
trainable_count = sum([1 for layer in resnet_model.layers if layer.trainable])
total_count = len(resnet_model.layers)

print("✅ ResNet50 créé")
print(f"   Couches totales : {total_count}")
print(f"   Couches entraînables : {trainable_count}")
print(f"   Couches gelées : {total_count - trainable_count}")
print(f"\n📊 Résumé :")
resnet_model.summary()

### 5.2 Compilation et entraînement

In [ ]:
# Compiler
resnet_model.compile(
    optimizer=Adam(learning_rate=1e-4),  # LR plus faible pour fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ ResNet50 compilé")
print("   Optimizer : Adam (lr=1e-4 pour fine-tuning)")

# Callbacks
resnet_callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        str(MODELS_DIR / 'resnet50_best.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("✅ Callbacks configurés")

In [ ]:
# Entraînement
print("\n🔄 Entraînement ResNet50 (fine-tuning)...")
print(f"⏳ {EPOCHS} epochs max\n")

history_resnet = resnet_model.fit(
    tf.data.Dataset.from_generator(
        train_gen,
        output_signature=(
            tf.TensorSpec(shape=(None, *IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, n_classes), dtype=tf.float32)
        )
    ).repeat(),
    steps_per_epoch=train_steps,
    epochs=EPOCHS,
    validation_data=tf.data.Dataset.from_generator(
        val_gen,
        output_signature=(
            tf.TensorSpec(shape=(None, *IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, n_classes), dtype=tf.float32)
        )
    ).repeat(),
    validation_steps=val_steps,
    callbacks=resnet_callbacks,
    verbose=1
)

print("\n✅ Entraînement ResNet50 terminé")

### 5.3 Courbes d'apprentissage

In [ ]:
# Visualisation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss
ax1.plot(history_resnet.history['loss'], label='Train Loss', linewidth=2)
ax1.plot(history_resnet.history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=11, fontweight='bold')
ax1.set_ylabel('Loss', fontsize=11, fontweight='bold')
ax1.set_title('ResNet50 — Loss', fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# Accuracy
ax2.plot(history_resnet.history['accuracy'], label='Train Accuracy', linewidth=2)
ax2.plot(history_resnet.history['val_accuracy'], label='Val Accuracy', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=11, fontweight='bold')
ax2.set_ylabel('Accuracy', fontsize=11, fontweight='bold')
ax2.set_title('ResNet50 — Accuracy', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Meilleures métriques
best_epoch_resnet = np.argmax(history_resnet.history['val_accuracy'])
best_val_acc_resnet = history_resnet.history['val_accuracy'][best_epoch_resnet]
best_val_loss_resnet = history_resnet.history['val_loss'][best_epoch_resnet]

print(f"\n🏆 Meilleures performances (epoch {best_epoch_resnet + 1}) :")
print(f"   Val Accuracy : {best_val_acc_resnet:.4f}")
print(f"   Val Loss     : {best_val_loss_resnet:.4f}")

### 5.4 Évaluation sur le test set

In [ ]:
# Évaluation
print("🔄 Évaluation ResNet50 sur le test set...\n")

test_loss_resnet, test_acc_resnet = resnet_model.evaluate(
    tf.data.Dataset.from_generator(
        test_gen,
        output_signature=(
            tf.TensorSpec(shape=(None, *IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, n_classes), dtype=tf.float32)
        )
    ),
    steps=test_steps
)

print(f"\n✅ ResNet50 — Test Set :")
print(f"   Loss     : {test_loss_resnet:.4f}")
print(f"   Accuracy : {test_acc_resnet:.4f} ({test_acc_resnet*100:.2f}%)")

## 6. MODÈLE 3 : TRANSFER LEARNING — EFFICIENTNETB0

### 6.1 Architecture

In [ ]:
def build_efficientnet_model(input_shape=(224, 224, 3), num_classes=7, trainable_layers=50):
    """
    EfficientNetB0 pré-entraîné avec fine-tuning
    """
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Geler les couches sauf les N dernières
    for layer in base_model.layers[:-trainable_layers]:
        layer.trainable = False
    
    # Couches de classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=output)
    
    return model, base_model

# Créer le modèle
efficientnet_model, efficientnet_base = build_efficientnet_model(
    input_shape=(*IMG_SIZE, 3),
    num_classes=n_classes,
    trainable_layers=50
)

trainable_eff = sum([1 for layer in efficientnet_model.layers if layer.trainable])
total_eff = len(efficientnet_model.layers)

print("✅ EfficientNetB0 créé")
print(f"   Couches totales : {total_eff}")
print(f"   Couches entraînables : {trainable_eff}")
print(f"   Couches gelées : {total_eff - trainable_eff}")

### 6.2 Compilation et entraînement

In [ ]:
# Compiler
efficientnet_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ EfficientNetB0 compilé")

# Callbacks
efficientnet_callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        str(MODELS_DIR / 'efficientnet_best.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("✅ Callbacks configurés")

In [ ]:
# Entraînement
print("\n🔄 Entraînement EfficientNetB0...")
print(f"⏳ {EPOCHS} epochs max\n")

history_efficientnet = efficientnet_model.fit(
    tf.data.Dataset.from_generator(
        train_gen,
        output_signature=(
            tf.TensorSpec(shape=(None, *IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, n_classes), dtype=tf.float32)
        )
    ).repeat(),
    steps_per_epoch=train_steps,
    epochs=EPOCHS,
    validation_data=tf.data.Dataset.from_generator(
        val_gen,
        output_signature=(
            tf.TensorSpec(shape=(None, *IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, n_classes), dtype=tf.float32)
        )
    ).repeat(),
    validation_steps=val_steps,
    callbacks=efficientnet_callbacks,
    verbose=1
)

print("\n✅ Entraînement EfficientNetB0 terminé")

### 6.3 Courbes et évaluation

In [ ]:
# Visualisation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(history_efficientnet.history['loss'], label='Train Loss', linewidth=2)
ax1.plot(history_efficientnet.history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=11, fontweight='bold')
ax1.set_ylabel('Loss', fontsize=11, fontweight='bold')
ax1.set_title('EfficientNetB0 — Loss', fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

ax2.plot(history_efficientnet.history['accuracy'], label='Train Accuracy', linewidth=2)
ax2.plot(history_efficientnet.history['val_accuracy'], label='Val Accuracy', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=11, fontweight='bold')
ax2.set_ylabel('Accuracy', fontsize=11, fontweight='bold')
ax2.set_title('EfficientNetB0 — Accuracy', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Évaluation test
print("\n🔄 Évaluation EfficientNetB0 sur le test set...\n")

test_loss_eff, test_acc_eff = efficientnet_model.evaluate(
    tf.data.Dataset.from_generator(
        test_gen,
        output_signature=(
            tf.TensorSpec(shape=(None, *IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, n_classes), dtype=tf.float32)
        )
    ),
    steps=test_steps
)

print(f"\n✅ EfficientNetB0 — Test Set :")
print(f"   Loss     : {test_loss_eff:.4f}")
print(f"   Accuracy : {test_acc_eff:.4f} ({test_acc_eff*100:.2f}%)")

## 7. COMPARAISON DES 3 MODÈLES

In [ ]:
# Tableau comparatif
results_dl = pd.DataFrame({
    'Modèle': ['CNN Custom', 'ResNet50', 'EfficientNetB0'],
    'Test Loss': [test_loss_cnn, test_loss_resnet, test_loss_eff],
    'Test Accuracy': [test_acc_cnn, test_acc_resnet, test_acc_eff]
})

results_dl = results_dl.sort_values('Test Accuracy', ascending=False).reset_index(drop=True)

print("\n" + "="*70)
print("📊 COMPARAISON DES MODÈLES DEEP LEARNING")
print("="*70)
print(results_dl.to_string(index=False))

# Meilleur modèle
best_model_name = results_dl.loc[0, 'Modèle']
best_model_acc = results_dl.loc[0, 'Test Accuracy']

print(f"\n🏆 MEILLEUR MODÈLE : {best_model_name}")
print(f"   Test Accuracy : {best_model_acc:.4f} ({best_model_acc*100:.2f}%)")

# Visualisation
fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.barh(results_dl['Modèle'], results_dl['Test Accuracy'], 
               color=['gold', 'steelblue', 'coral'][:len(results_dl)],
               edgecolor='black', alpha=0.8)

ax.set_xlabel('Test Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Comparaison des modèles Deep Learning', fontsize=13, fontweight='bold')
ax.set_xlim(0, 1)
ax.grid(axis='x', alpha=0.3)

# Valeurs sur les barres
for bar in bars:
    width = bar.get_width()
    ax.text(width + 0.01, bar.get_y() + bar.get_height()/2.,
            f'{width:.4f}',
            ha='left', va='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## 8. ANALYSE DÉTAILLÉE DU MEILLEUR MODÈLE

### 8.1 Prédictions et métriques complètes

In [ ]:
# Sélectionner le meilleur modèle
if best_model_name == 'CNN Custom':
    best_model = cnn_model
elif best_model_name == 'ResNet50':
    best_model = resnet_model
else:
    best_model = efficientnet_model

# Faire des prédictions sur le test set
print("🔄 Génération des prédictions sur le test set...\n")

y_true_list = []
y_pred_list = []

for batch_X, batch_y in tf.data.Dataset.from_generator(
    test_gen,
    output_signature=(
        tf.TensorSpec(shape=(None, *IMG_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, n_classes), dtype=tf.float32)
    )
).take(test_steps):
    preds = best_model.predict(batch_X, verbose=0)
    y_true_list.extend(np.argmax(batch_y.numpy(), axis=1))
    y_pred_list.extend(np.argmax(preds, axis=1))

y_true_arr = np.array(y_true_list)
y_pred_arr = np.array(y_pred_list)

print(f"✅ Prédictions générées : {len(y_true_arr)} échantillons")

# Métriques détaillées
test_precision = precision_score(y_true_arr, y_pred_arr, average='weighted')
test_recall = recall_score(y_true_arr, y_pred_arr, average='weighted')
test_f1 = f1_score(y_true_arr, y_pred_arr, average='weighted')

print(f"\n📊 MÉTRIQUES COMPLÈTES — {best_model_name}")
print("="*70)
print(f"   Accuracy  : {best_model_acc:.4f} ({best_model_acc*100:.2f}%)")
print(f"   Precision : {test_precision:.4f}")
print(f"   Recall    : {test_recall:.4f}")
print(f"   F1-Score  : {test_f1:.4f}")

### 8.2 Rapport de classification par catégorie

In [ ]:
print("\n" + "="*70)
print(f"📊 RAPPORT DE CLASSIFICATION PAR CATÉGORIE — {best_model_name}")
print("="*70)
print(classification_report(y_true_arr, y_pred_arr, target_names=le.classes_))

### 8.3 Matrice de confusion

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_true_arr, y_pred_arr)

# Brute
plt.figure(figsize=(11, 9))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_,
            yticklabels=le.classes_,
            cbar_kws={'label': 'Nombre'},
            linewidths=0.5)
plt.xlabel('Classe prédite', fontsize=12, fontweight='bold')
plt.ylabel('Classe réelle', fontsize=12, fontweight='bold')
plt.title(f'Matrice de confusion — {best_model_name}', 
          fontsize=13, fontweight='bold', pad=15)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Normalisée
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(11, 9))
sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Greens',
            xticklabels=le.classes_,
            yticklabels=le.classes_,
            cbar_kws={'label': 'Proportion'},
            linewidths=0.5,
            vmin=0, vmax=1)
plt.xlabel('Classe prédite', fontsize=12, fontweight='bold')
plt.ylabel('Classe réelle', fontsize=12, fontweight='bold')
plt.title(f'Matrice de confusion normalisée — {best_model_name}', 
          fontsize=13, fontweight='bold', pad=15)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## 9. RÉCAPITULATIF FINAL

### 9.1 Résultats du modèle IMAGES (Deep Learning)

In [ ]:
print("\n" + "="*80)
print("📊 RÉCAPITULATIF — CLASSIFICATION D'IMAGES PAR DEEP LEARNING")
print("="*80)

print("\n🔹 DONNÉES :")
print(f"   {len(df)} images réparties en {n_classes} catégories")
print(f"   Split : 70% train / 15% val / 15% test")
print(f"   Images size : {IMG_SIZE}")

print("\n🔹 AUGMENTATION DE DONNÉES :")
print("   Rotation, translation, flip, zoom, shear")

print("\n🔹 MODÈLES TESTÉS :")
print("   1. CNN Custom (from scratch)")
print("   2. ResNet50 (Transfer Learning + fine-tuning)")
print("   3. EfficientNetB0 (Transfer Learning + fine-tuning)")

print(f"\n🔹 MEILLEUR MODÈLE :")
print(f"   {best_model_name}")
print(f"   Test Accuracy : {best_model_acc:.4f} ({best_model_acc*100:.2f}%)")
print(f"   Precision     : {test_precision:.4f}")
print(f"   Recall        : {test_recall:.4f}")
print(f"   F1-Score      : {test_f1:.4f}")

print("\n🔹 OBSERVATIONS :")
if best_model_acc > 0.80:
    print("   ✅ Excellentes performances — modèle prêt pour la production")
elif best_model_acc > 0.70:
    print("   ✅ Bonnes performances — modèle utilisable en production")
elif best_model_acc > 0.60:
    print("   ⚠️ Performances modérées — améliorations possibles")
else:
    print("   ⚠️ Performances faibles — augmentation données ou architectures plus complexes nécessaires")

### 9.2 Résultats du modèle TEXTE (Notebook 3)

**⚠️ IMPORTANT** : Remplacez ces valeurs par vos résultats réels du notebook 3

In [ ]:
# ⬇️ REMPLACER CES VALEURS PAR VOS RÉSULTATS RÉELS DU NOTEBOOK 3
text_results = {
    'Modèle': 'SVM (TF-IDF)',  # ⬅️ ADAPTER
    'Accuracy': 0.9476,         # ⬅️ REMPLACER
    'Precision': 0.9450,        # ⬅️ REMPLACER
    'Recall': 0.9432,           # ⬅️ REMPLACER
    'F1-Score': 0.9432          # ⬅️ REMPLACER
}

print("="*80)
print("📊 RÉCAPITULATIF — CLASSIFICATION À PARTIR DU TEXTE (Notebook 3)")
print("="*80)

print("\n🔹 MEILLEUR MODÈLE TEXTE :")
print(f"   {text_results['Modèle']}")
print(f"   Accuracy  : {text_results['Accuracy']:.4f} ({text_results['Accuracy']*100:.2f}%)")
print(f"   Precision : {text_results['Precision']:.4f}")
print(f"   Recall    : {text_results['Recall']:.4f}")
print(f"   F1-Score  : {text_results['F1-Score']:.4f}")

print("\n💡 Ces valeurs proviennent du Notebook 3 (features textuelles TF-IDF)")

### 9.3 Tableau comparatif final TEXTE vs IMAGES

In [ ]:
# Tableau récapitulatif
recap_final = pd.DataFrame([
    {
        'Modalité': 'TEXTE (descriptions)',
        'Meilleur modèle': text_results['Modèle'],
        'Accuracy': text_results['Accuracy'],
        'Precision': text_results['Precision'],
        'Recall': text_results['Recall'],
        'F1-Score': text_results['F1-Score']
    },
    {
        'Modalité': 'IMAGES (Deep Learning)',
        'Meilleur modèle': best_model_name,
        'Accuracy': best_model_acc,
        'Precision': test_precision,
        'Recall': test_recall,
        'F1-Score': test_f1
    }
])

print("\n" + "="*80)
print("🎯 TABLEAU RÉCAPITULATIF FINAL : TEXTE vs IMAGES")
print("="*80)
print(recap_final.to_string(index=False))

# Sauvegarder
recap_csv = RESULTS_DIR / "recap_final_texte_vs_images.csv"
recap_final.to_csv(recap_csv, index=False)
print(f"\n💾 Tableau sauvegardé : {recap_csv}")

In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['#4CAF50', '#2196F3']  # Vert texte, bleu images

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    
    values = recap_final[metric].values
    labels = ['Texte', 'Images']
    
    bars = ax.bar(labels, values, color=colors, 
                  edgecolor='black', alpha=0.85, width=0.6)
    
    ax.set_ylabel(metric, fontsize=12, fontweight='bold')
    ax.set_ylim(0, 1)
    ax.set_title(f'{metric} : Texte vs Images (Deep Learning)', 
                 fontsize=13, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Valeurs sur barres
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{height:.4f}',
                ha='center', va='bottom', 
                fontsize=11, fontweight='bold')

plt.suptitle('Comparaison finale : Modèle TEXTE vs Modèle IMAGES (Deep Learning)', 
             fontsize=15, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

### 9.4 Conclusions finales et recommandations

In [ ]:
print("\n" + "="*80)
print("🎓 CONCLUSIONS FINALES ET RECOMMANDATIONS")
print("="*80)

diff_acc = text_results['Accuracy'] - best_model_acc
diff_f1 = text_results['F1-Score'] - test_f1

print("\n🔹 COMPARAISON DES PERFORMANCES :")
print(f"\n   Modèle TEXTE ({text_results['Modèle']}) :")
print(f"   → Accuracy : {text_results['Accuracy']:.4f} ({text_results['Accuracy']*100:.2f}%)")
print(f"   → F1-Score : {text_results['F1-Score']:.4f}")

print(f"\n   Modèle IMAGES ({best_model_name}) :")
print(f"   → Accuracy : {best_model_acc:.4f} ({best_model_acc*100:.2f}%)")
print(f"   → F1-Score : {test_f1:.4f}")

print(f"\n   Différence Accuracy : {diff_acc:+.4f} ({diff_acc*100:+.2f} points)")
print(f"   Différence F1-Score : {diff_f1:+.4f} ({diff_f1*100:+.2f} points)")

print("\n🔹 RECOMMANDATIONS POUR LA PRODUCTION :")

if diff_f1 > 0.15:
    print("\n   🎯 STRATÉGIE : Privilégier le modèle TEXTE")
    print("   1. ✅ Utiliser le modèle TEXTE comme modèle principal")
    print("   2. 💡 Modèle IMAGES comme fallback si description manquante/courte")
    print("   3. 🚀 Envisager fusion multimodale pour maximiser la performance")
elif diff_f1 > 0.05:
    print("\n   🎯 STRATÉGIE : Combiner les deux modalités")
    print("   1. 🚀 Implémenter une Late Fusion (moyenne pondérée des prédictions)")
    print("   2. 💡 Poids suggérés : 70% texte, 30% images (à optimiser)")
    print("   3. ✅ Utiliser texte seul si description riche, images seules sinon")
elif diff_f1 > -0.05:
    print("\n   🎯 STRATÉGIE : Performances similaires")
    print("   1. 🔄 Choisir selon le contexte (texte riche vs images de qualité)")
    print("   2. 🚀 Fusion multimodale recommandée pour maximiser")
    print("   3. ⚡ Modèle IMAGES plus rapide en inférence que texte+vectorisation")
else:
    print("\n   🎯 STRATÉGIE : Privilégier le modèle IMAGES")
    print("   1. ✅ Utiliser le modèle IMAGES (Deep Learning) comme principal")
    print("   2. 💡 Modèle TEXTE comme fallback ou pour validation")
    print("   3. 🚀 Fusion multimodale pourrait encore améliorer")

print("\n🔹 AMÉLIORATIONS POSSIBLES :")
print("\n   📸 Pour les IMAGES (Deep Learning) :")
print("      - Plus d'époques d'entraînement (50 → 100)")
print("      - Data augmentation plus agressive")
print("      - Tester d'autres architectures (Vision Transformer, ConvNeXt, Swin)")
print("      - Ensembling (moyenne de plusieurs modèles)")
print("      - Pseudo-labeling sur données non étiquetées")

print("\n   📝 Pour le TEXTE :")
print("      - Embeddings pré-entraînés (BERT, USE, Sentence-BERT)")
print("      - Modèles de langue (LSTM, Transformer)")
print("      - Augmentation de données textuelles (back-translation)")

print("\n   🔗 Pour la FUSION MULTIMODALE :")
print("      - Late Fusion : moyenne pondérée des softmax")
print("      - Intermediate Fusion : concaténer features texte + images → MLP")
print("      - Early Fusion : architecture end-to-end (CNN + LSTM)")
print("      - Attention mechanism pour pondération dynamique")

print("\n" + "="*80)
print("✅ NOTEBOOK 5 TERMINÉ")
print("="*80)
print("\n🎉 Projet de classification de produits e-commerce achevé !")
print(f"📊 Modèles sauvegardés dans : {MODELS_DIR}")
print(f"📊 Résultats dans : {RESULTS_DIR}")